In [3]:
# auto reload the Python source in ../src/anndata_repr/__init__.py without needing to restart kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# retrieve dataset
from anndata_repr import read_h5ad
adata = read_h5ad("../data/scverse-getting-started-anndata-pbmc3k_processed.h5ad")
adata

AnnData object with n_obs × n_vars = 2638 × 11505
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain_cell_types'
    var: 'gene_names', 'n_cells', 'gene_ids'
    uns: 'louvain', 'louvain_colors', 'pca'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'raw'
    obsp: 'distances_all'